# Step 1: Set up D3 area

In [1]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [2]:
HTML('<script src="lib/d3/d3.min.js"></script>')

In [47]:
html_template = Template('''
<svg id="graph-div"><g class="item"><rect x="50" y="50" height="100" width="100" style="stroke:#000; fill: #0086B2"></rect><text x="50" y="50">Hello World</text></g></div>
<script> $js_text </script>
''')

In [48]:
js_text_template = Template('''
var data = $data;

var chart = d3.select('#graph-div')
    .attr('width', 400)
    .attr('height', 400);

function update() {
    var row = chart.select('.item')
        .attr('transform', function(d, i){
            return 'rotate(' + data + ', 100, 100)';
        });
}

update();
''')

In [49]:
data = 0;
js_text = js_text_template.substitute({'data': json.dumps(data)})
HTML(html_template.substitute({'js_text': js_text}))

### OK, the D3 area is set up

Now we'll focus on live updating.  A manual test first.

In [50]:
js_text_template_2 = Template('''
data = $data;
update();
console.log("updating");
''')

In [51]:
data = 0 #[0,500,1000,2000,4000]

def update_graph(data):
    js_text = js_text_template_2.substitute({'data': json.dumps(data)})
    display(HTML('<script>' + js_text + '</script>'))

update_graph(data)

# Step 2: Now use MQTT to update the graph

Now for the fun stuff.  Using the `update_graph(data)` function set up above.  Each group of data, separated by `"start"` messages, is saved in the `tests` array.  As new data comes in, the live data graph is updated to show data from the last test.  All data is neatly saved inside the tests array for replotting and further analysis later.

Using the CloudMQTT free Cat plan:
* https://api.cloudmqtt.com/sso/cloudmqtt/console
* haipjacob@gmail.com:paramour-sieve-paper
* If the Websocket UI has trouble connecting, restart the instance

In [52]:
import paho.mqtt.client as mqtt

data = []

# update_graph(data)

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("/outTopic")

def on_message(client, userdata, msg):
    global data
    try:
        msg_json = json.loads(msg.payload)
    except:
        print "Error"
        print(msg.topic+" "+str(msg.payload))
        return
    # print msg_json
    if msg_json['type'] == "BINARY" and msg_json['label'] == "Or":
        data.append((msg_json['X'], msg_json['Y'], msg_json['Z']))
        update_graph(msg_json['Z'])
        print msg_json['Z']

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.username_pw_set("zettlmtm", "VOUbRcmhjffA")
client.connect("m11.cloudmqtt.com", 19280, 60)
client.loop_start()
# Make sure to call client.loop_stop() later

Connected with result code 0


1.25


1.25


1.25


1.25


...

1.0


1.0


1.0


1.0


1.0


In [54]:
client.loop_stop()

3

In [55]:
print data

[(168.12, -0.62, 1.25), (168.12, -0.62, 1.25), (168.12, -0.62, 1.25), (168.12, -0.62, 1.25), (168.12, -0.62, 1.25), (168.12, -0.62, 1.25), (168.12, -0.62, 1.25), (168.12, -0.62, 1.25), (167.19, -0.25, -3.44), (169.44, 0.69, -12.44), (173.12, 0.62, -22.0), (178.88, -0.31, -46.88), (177.12, 0.38, -74.69), (176.5, 0.5, -81.0), (178.19, 1.25, -83.5), (180.5, 1.0, -41.06), (182.19, 2.31, -3.56), (178.62, 3.38, 38.38), (173.38, -1.38, 71.38), (171.75, -2.56, 82.75), (173.75, -3.94, 85.69), (179.44, -5.06, 77.88), (177.81, -29.5, 43.94), (182.81, -60.25, 27.62), (186.56, -72.81, 20.69), (188.56, -67.44, 21.38), (188.75, -26.56, 22.69), (181.94, -18.75, 73.19), (182.38, -13.62, 127.88), (31.81, -4.5, 155.94), (30.56, 0.06, 162.62), (30.12, 2.81, 150.06), (212.5, 1.5, 107.19), (205.19, -2.88, 56.88), (199.38, -0.81, 8.5), (197.38, 2.38, -33.56), (195.62, 7.62, -72.31), (191.19, 8.94, -105.81), (193.69, 6.56, -128.31), (33.81, 8.69, -150.5), (36.62, 9.75, -155.5), (38.56, 9.44, -158.06), (36.75,

# Results were good!

The Particle Build IDE had the Adafruit Sensor Library but was missing the Adafruit_BNO055 library that was mentioned in the [tutorial](https://learn.adafruit.com/adafruit-bno055-absolute-orientation-sensor/wiring-and-test).  Thankfully I found a [port](https://github.com/ghirlekar/photon_BNO055) publicly available on Github via @ghirlekar.

I did run in to an issue with the firmware where `euler.x()` and the like were returning floats but the `sprintf` function I had copied from other MQTT logging programs was expecting a decimal.  Because of this the return values were almost always 0.  The second issue was the MQTT message string was too long on the firmware because I had a set character limit of 100.  I changed the labels so the logged message was shorter and the problem went away.  The problem wasn't reported by the compiler but when running the Photon would sent the "Hello World" MQTT message and then the on-board light would flash red a couple times.  I also had to play with the delay time of sending messages.  It appeared if they were sent too quickly the D3 viz in this notebook would lag and freeze for a few seconds.  Eventually I settled on 200ms delay between messages.  The root cause of the delay could be the dropped messages from MQTT and the CloudMQTT broker or and issue with this notebook processing all of them fast enough.

I remembered from before that the sensor readings get better if you hold the 9DOF IMU in different orientations for a few seconds each to help the sensors self calibrate.  I was surpised in the end the results were very stable and worked well 360 degrees (at least for the Z axis which was the only one I visualized).

The current demo only show the Z rotation because SVG can only do rotation in one dimension and I didn't want to fiddle too much with getting full XYZ rotation.  The `euler.x()` results from the firwmare library return degrees from -180 to 180.
